In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership (
  store_id STRING,
  calday DATE,
  csn STRING,
  customer_id STRING,
  user_id STRING,
  fullname STRING,
  staff_id STRING,
  merchantcustomerid STRING,
  mobile_number STRING,
  gender STRING,
  dob DATE,
  fullname_enc BINARY,
  staff_id_enc BINARY,
  merchantcustomerid_enc BINARY,
  mobile_number_enc BINARY,
  gender_enc BINARY,
  dob_enc BINARY,
  reg_source STRING,
  merchantstoreid STRING,
  register_channel STRING,
  masan_referral_code STRING,
  source_data STRING NOT NULL,
  mobile_enroll STRING,
  marketing_noti STRING,
  pilot_store_id STRING,
  pos_enroll STRING,
  onboarding_store_id STRING,
  mobile_enroll_date STRING,
  mobile_noti STRING,
  policy_items STRING,
  device STRING,
  pilot_group STRING,
  loyalty_points DOUBLE,
  lifetime_points DOUBLE,
  redeemed_points DOUBLE,
  expired_points DOUBLE,
  is_vinid STRING NOT NULL,
  reg_source_biz STRING,
  capilary_customer_id STRING,
  b_staff_id STRING,
  is_employee BOOLEAN NOT NULL
)
USING delta
TBLPROPERTIES (
  'delta.autoOptimize.autoCompact' = 'true',
  'delta.autoOptimize.optimizeWrite' = 'true'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW temp_d_win_membership AS
WITH
member_register_prepare AS (
  SELECT
      CASE
          WHEN a.source IN ('PLH-WEB', 'PLH-POS') THEN 'MASANPHUCLONGECOMPROD'
          ELSE coalesce(c.store_id, CONCAT(a.source, "__", a.merchant_store_id))
      END AS store_id,
      TO_DATE(try_to_timestamp(a.create_date) + INTERVAL 7 HOURS) as calday,
      a.csn,
      e.id AS customer_id,
      CAST(NULL AS STRING) AS user_id,
      e.full_name as fullname,
      a.ref_staff_id AS staff_id,
      a.merchant_customer_id as merchantcustomerid,
      {catalog_name}.default.fn_clean_mobile_no(a.merchant_customer_id) AS mobile_number,
      e.gender,
      IF(e.dob = 'Invalid date' OR e.dob IS NULL, NULL, TRY_CAST(LEFT(e.dob, 10) AS DATE)) dob,
      UNHEX(MD5(e.full_name)) fullname_enc,
      UNHEX(MD5(a.ref_staff_id)) staff_id_enc,
      UNHEX(MD5(a.merchant_customer_id)) merchantcustomerid_enc,
      UNHEX(MD5({catalog_name}.default.fn_clean_mobile_no(a.merchant_customer_id))) AS mobile_number_enc,
      UNHEX(MD5(e.gender)) gender_enc,
      UNHEX(MD5(IF(e.Dob = 'Invalid date' OR e.Dob IS NULL, NULL, LEFT(e.Dob, 10)))) AS dob_enc,
      source AS reg_source,
      a.merchant_store_id AS merchantstoreid,
      CASE
          WHEN a.source LIKE 'WMT-WEB%' AND a.merchant_store_id = 'VCM' AND a.ref_staff_id LIKE '%TCB%' THEN 'winmart TCB đk'
          WHEN a.source LIKE 'WMT-WEB%' AND a.merchant_store_id = 'VCM' AND IFNULL(a.ref_staff_id,'NULL') NOT LIKE '%TCB%' THEN 'winmart.vn KH/WCM đk'
          ELSE IFNULL(d.reg_source_name_group_lv2, CONCAT(source, "-*-", a.merchant_store_id))
      END AS register_channel,
      CAST(NULL AS STRING) masan_referral_code,
      'CX' AS source_data,
      CAST(NULL AS STRING) mobile_enroll,
      CAST(NULL AS STRING) marketing_noti,
      CAST(NULL AS STRING) pilot_store_id,
      CAST(NULL AS STRING) pos_enroll,
      CAST(NULL AS STRING) onboarding_store_id,
      CAST(NULL AS STRING) mobile_enroll_date,
      CAST(NULL AS STRING) mobile_noti,
      CAST(NULL AS STRING) policy_items,
      CAST(NULL AS STRING) device,
      CAST(NULL AS STRING) pilot_group,
      CAST(NULL AS DOUBLE) loyalty_points,
      CAST(NULL AS DOUBLE) lifetime_points,
      CAST(NULL AS DOUBLE) redeemed_points,
      CAST(NULL AS DOUBLE) expired_points,
      3 priority
  FROM
      {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_customer_merchant_rels a
      INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time b
        ON TO_DATE(try_to_timestamp(a.create_date) + INTERVAL 7 HOURS) = b.calday
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c
        ON IFNULL(a.store_no, 'null') = c.store_id
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.cfg_mbs_register_source_mapping d
        ON a.source = d.source_ AND a.merchant_store_id = d.merchantstoreid
      LEFT JOIN {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_loyalty_customers e ON a.csn = e.csn
  WHERE
      a.merchant_id = 'WIN'
    AND DATE(try_to_timestamp(a.create_date) + INTERVAL 7 HOURS) <= DATE('2023-12-05')
    AND e.is_deleted <> 'true'
    AND a.is_deleted <> 'true'
  UNION ALL
      -- từ ngày 6/12 đến ngày d-2  lấy dữ liệu ở bảng daily
  SELECT
        CASE
                WHEN a.store_id = 'MASANWINCAREPROD'  and a.masan_referral_code is not null and a.masan_referral_code <> "" THEN a.masan_referral_code
                ELSE a.store_id
        END store_id,
        a.calday,
        CAST(NULL AS STRING) AS csn,
        a.customer_id,
        -- Bổ sung UserID bên CAPI
        a.user_id,
        a.fullname,
        a.staff_id,
        CAST(a.merchant_customer_id AS STRING) as merchantcustomerid,
        {catalog_name}.default.fn_clean_mobile_no(a.mobile)  mobile_number,
        CASE
            WHEN UPPER(a.gender) = 'MALE' THEN 'M'
            WHEN UPPER(a.gender) = 'FEMALE' THEN 'F'
        END gender,
        IF(a.dob = 'Invalid date' OR a.dob IS NULL, NULL, TRY_CAST(LEFT(a.dob, 10) AS DATE)) dob,
        UNHEX(MD5(a.fullname)) fullname_enc,
        UNHEX(MD5(a.staff_id)) staff_id_enc,
        UNHEX(MD5(CAST(a.merchant_customer_id AS STRING))) merchantcustomerid_enc,
        UNHEX(MD5({catalog_name}.default.fn_clean_mobile_no(a.mobile))) AS mobile_number_enc,
        UNHEX(MD5(CASE
            WHEN UPPER(a.gender) = 'MALE' THEN 'M'
            WHEN UPPER(a.gender) = 'FEMALE' THEN 'F'
        END)) gender_enc,
        UNHEX(MD5(CAST(a.dob AS STRING))) AS dob_enc,
        a.reg_source,
        a.merchant_store_id as merchantstoreid,
        CASE
          WHEN a.reg_source LIKE 'WMT-WEB%' AND a.merchant_store_id = 'VCM' AND a.staff_id LIKE '%TCB%' THEN 'winmart TCB đk'
          WHEN a.reg_source LIKE 'WMT-WEB%' AND a.merchant_store_id = 'VCM' AND IFNULL(a.staff_id,'NULL') NOT LIKE '%TCB%' THEN 'winmart.vn KH/WCM đk'
          ELSE IFNULL(d.reg_source_name_group_lv2, CONCAT(a.reg_source, "-*-", a.merchant_store_id))
      END AS register_channel,
      a.masan_referral_code,
      'CAPILLARY' AS source_data,
      mobile_enroll,
      marketing_noti,
      pilot_store_id,
      pos_enroll,
      onboarding_store_id,
      mobile_enroll_date,
      mobile_noti,
      policy_items,
      device,
      pilot_group,
      loyalty_points,
      lifetime_points,
      redeemed_points,
      expired_points,
      1 priority
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_win_membership a -- daily from CAPILLARY
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.cfg_mbs_register_source_mapping d
        ON a.reg_source = d.source_ AND a.merchant_store_id = d.merchantstoreid
  WHERE a.calday <= DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1)
    AND a.is_membership = 1
  QUALIFY ROW_NUMBER() OVER (PARTITION BY {catalog_name}.default.fn_clean_mobile_no(a.mobile) ORDER BY a.proc_date DESC) = 1
  UNION ALL
-- NGÀY D-1  lấy dữ liệu ở bảng REALTIME CAP
  SELECT
        CASE
                WHEN a.store_id = 'MASANWINCAREPROD' AND a.masan_referral_code IS NOT NULL AND a.masan_referral_code <> "" THEN a.masan_referral_code
                ELSE a.store_id
        END store_id,
        a.calday,
        CAST(NULL AS STRING) AS csn,
        a.customer_id,
        a.user_id,
        a.fullname,
        a.staff_id,
        CAST(a.merchant_customer_id AS STRING) AS merchantcustomerid,
        {catalog_name}.default.fn_clean_mobile_no(a.mobile_number) mobile_number,
        CASE
            WHEN UPPER(a.gender) = 'MALE' THEN 'M'
            WHEN UPPER(a.gender) = 'FEMALE' THEN 'F'
        END gender,
        a.dob,
        UNHEX(MD5(a.fullname)) fullname_enc,
        UNHEX(MD5(a.staff_id)) staff_id_enc,
        UNHEX(MD5(CAST(a.merchant_customer_id AS STRING))) merchantcustomerid_enc,
        UNHEX(MD5({catalog_name}.default.fn_clean_mobile_no(a.mobile_number))) AS mobile_number_enc,
        UNHEX(MD5(CASE
            WHEN UPPER(a.gender) = 'MALE' THEN 'M'
            WHEN UPPER(a.gender) = 'FEMALE' THEN 'F'
        END)) gender_enc,
        UNHEX(MD5(CAST(a.dob AS STRING))) AS dob_enc,
        a.reg_source,
        a.merchant_store_id as merchantstoreid,
        CASE
          WHEN a.reg_source LIKE 'WMT-WEB%' AND a.merchant_store_id = 'VCM' AND a.staff_id LIKE '%TCB%' THEN 'winmart TCB đk'
          WHEN a.reg_source LIKE 'WMT-WEB%' AND a.merchant_store_id = 'VCM' AND IFNULL(a.staff_id,'NULL') NOT LIKE '%TCB%' THEN 'winmart.vn KH/WCM đk'
          ELSE IFNULL(d.reg_source_name_group_lv2, CONCAT(a.reg_source, "-*-", a.merchant_store_id))
      END AS register_channel,
      CAST(NULL AS STRING) masan_referral_code,
      'CAPILLARY' AS source_data,
      CAST(NULL AS STRING) mobile_enroll,
      CAST(NULL AS STRING) marketing_noti,
      CAST(NULL AS STRING) pilot_store_id,
      CAST(NULL AS STRING) pos_enroll,
      CAST(NULL AS STRING) onboarding_store_id,
      CAST(NULL AS STRING) mobile_enroll_date,
      CAST(NULL AS STRING) mobile_noti,
      CAST(NULL AS STRING) policy_items,
      CAST(NULL AS STRING) device,
      CAST(NULL AS STRING) pilot_group,
      CAST(NULL AS DOUBLE) loyalty_points,
      CAST(NULL AS DOUBLE) lifetime_points,
      CAST(NULL AS DOUBLE) redeemed_points,
      CAST(NULL AS DOUBLE) expired_points,
      2 priority
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_membership a -- REALTIME from CAPILLARY
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.cfg_mbs_register_source_mapping d
        ON a.reg_source = d.source_ AND a.merchant_store_id = d.merchantstoreid
  WHERE a.calday = DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1)
  QUALIFY ROW_NUMBER() OVER (PARTITION BY {catalog_name}.default.fn_clean_mobile_no(a.mobile_number) ORDER BY a.proc_date DESC) = 1
),
member_register AS (
  SELECT * EXCEPT(priority)
  FROM (
    SELECT * FROM member_register_prepare
    QUALIFY DENSE_RANK() OVER (PARTITION BY calday ORDER BY priority) = 1
  )
  QUALIFY ROW_NUMBER() OVER (PARTITION BY mobile_number ORDER BY priority ASC) = 1
),
vinid_cus AS (
  SELECT
    DISTINCT {catalog_name}.default.fn_clean_mobile_no(cus.mobile_number) mobile_number
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.view_d_vinid_customers_full cus
),
d_store AS (
  SELECT store_id,
          CASE
              WHEN business_unit = '1500' THEN 'WMT'
              WHEN business_unit = '2000' AND concept = 'WiN' THEN 'WIN'
              WHEN business_unit = '2000' AND concept = 'Rural' THEN 'RURAL'
              WHEN business_unit = '2000' THEN 'URBAN'
          END concept
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
),
customerid_cap_daily AS (
  SELECT {catalog_name}.default.fn_clean_mobile_no(mobile) mobile_number,
          user_id customer_id
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_win_membership
  QUALIFY ROW_NUMBER() OVER(PARTITION BY {catalog_name}.default.fn_clean_mobile_no(mobile) ORDER BY 1) = 1
),
customerid_cap_realtime AS (
  SELECT  {catalog_name}.default.fn_clean_mobile_no(mobile_number) mobile_number,
          user_id customer_id
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_capillary_membership
  QUALIFY ROW_NUMBER() OVER(PARTITION BY {catalog_name}.default.fn_clean_mobile_no(mobile_number) ORDER BY 1) = 1
),
final AS (
  SELECT a.*,
      CASE WHEN c.mobile_number IS NOT NULL THEN 'YES' ELSE 'NO' END is_vinid,
      CASE
          WHEN a.staff_id LIKE '%TCB%' THEN 'TCB'
          WHEN a.staff_id IS NULL THEN 'OTHER'
          ELSE 'OTHER'
      END staff_id_map,
      CASE
          WHEN a.reg_source = 'POS' AND (a.merchantstoreid NOT IN ('PLH', 'DWN', 'VCM') OR a.merchantstoreid IS NULL) THEN 'OTHER'
          WHEN a.reg_source = 'WINCARE' AND (a.merchantstoreid NOT IN ('TCB', 'VCM') OR a.merchantstoreid IS NULL) THEN 'OTHER'
          ELSE a.merchantstoreid
      END merchantstoreid_map,
      d_store.concept
  FROM member_register a
  LEFT JOIN vinid_cus c ON a.mobile_number = c.mobile_number
  LEFT JOIN d_store ON a.store_id = d_store.store_id
)
SELECT 
    a.* EXCEPT(concept, merchantstoreid_map, staff_id_map),
    CASE
        WHEN b.final_source IS NOT NULL THEN b.final_source
        ELSE 'Others'
    END AS reg_source_biz,
    CASE
        WHEN daily.customer_id IS NOT NULL THEN daily.customer_id
        WHEN rt.customer_id IS NOT NULL THEN rt.customer_id
        ELSE a.customer_id
    END capilary_customer_id,
    b.staff_id AS b_staff_id,
    (c.mobile_number IS NOT NULL) is_employee
FROM final a
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_employment_info c ON a.mobile_number = c.mobile_number
LEFT JOIN customerid_cap_daily daily ON a.mobile_number = daily.mobile_number
LEFT JOIN customerid_cap_realtime rt ON a.mobile_number = rt.mobile_number
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_reg_source_membership b 
ON a.reg_source = b.reg_source
AND (
    CASE
        WHEN b.merchantstoreid IS NULL THEN TRUE 
        WHEN b.merchantstoreid IS NOT NULL THEN a.merchantstoreid_map = b.merchantstoreid
    END
)
AND (
    CASE
        WHEN b.staff_id IS NULL THEN TRUE
        WHEN b.staff_id IS NOT NULL THEN b.staff_id = a.staff_id_map
    END
)
AND (
    CASE
        WHEN b.concept IS NULL THEN TRUE
        WHEN b.concept IS NOT NULL THEN b.concept = a.concept
    END
);
""")

In [0]:
spark.sql(f"""
TRUNCATE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership
SELECT * FROM temp_d_win_membership
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership_his (
  data_day DATE,
  store_id STRING,
  calday DATE,
  csn STRING,
  customer_id STRING,
  user_id STRING,
  fullname STRING,
  staff_id STRING,
  merchantcustomerid STRING,
  mobile_number STRING,
  gender STRING,
  dob DATE,
  fullname_enc BINARY,
  staff_id_enc BINARY,
  merchantcustomerid_enc BINARY,
  mobile_number_enc BINARY,
  gender_enc BINARY,
  dob_enc BINARY,
  reg_source STRING,
  merchantstoreid STRING,
  register_channel STRING,
  masan_referral_code STRING,
  source_data STRING NOT NULL,
  mobile_enroll STRING,
  marketing_noti STRING,
  pilot_store_id STRING,
  pos_enroll STRING,
  onboarding_store_id STRING,
  mobile_enroll_date STRING,
  mobile_noti STRING,
  policy_items STRING,
  device STRING,
  pilot_group STRING,
  loyalty_points DOUBLE,
  lifetime_points DOUBLE,
  redeemed_points DOUBLE,
  expired_points DOUBLE,
  is_vinid STRING NOT NULL,
  reg_source_biz STRING,
  capilary_customer_id STRING,
  b_staff_id STRING,
  is_employee BOOLEAN NOT NULL
)
USING delta
TBLPROPERTIES (
  'delta.autoOptimize.autoCompact' = 'true',
  'delta.autoOptimize.optimizeWrite' = 'true'
)
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership_his 
WHERE data_day = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
""")
  

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership_his 
SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AS data_day,
    *
FROM temp_d_win_membership;
""")